In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [86]:
out='../output/'
inp='../input/monthly/'
pref='fpm'

In [87]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,Min,,
2,Max,,
3,Average,,
4,Total,,
...,...,...,...
720,(2022/09/30) 23-24,45.05,20
721,Min,45.05,0
722,Max,90,20
723,Average,66.898611111111111111111111111,9.444444444444444444444444444


In [88]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
5,(2017/04/01) 04-05,39.06,9.4
6,(2017/04/01) 05-06,39.06,9.4
7,(2017/04/01) 06-07,39.06,9.4
8,(2017/04/01) 07-08,67.32,0.0
9,(2017/04/01) 08-09,67.32,0.0
...,...,...,...
716,(2022/09/30) 19-20,90.00,0.0
717,(2022/09/30) 20-21,85.00,0.0
718,(2022/09/30) 21-22,85.00,0.0
719,(2022/09/30) 22-23,45.05,20.0


In [89]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
5,(2017/04/01) 04-05,39.06,9.4,2017-04-01,4
6,(2017/04/01) 05-06,39.06,9.4,2017-04-01,5
7,(2017/04/01) 06-07,39.06,9.4,2017-04-01,6
8,(2017/04/01) 07-08,67.32,0.0,2017-04-01,7
9,(2017/04/01) 08-09,67.32,0.0,2017-04-01,8
...,...,...,...,...,...
716,(2022/09/30) 19-20,90.00,0.0,2022-09-30,19
717,(2022/09/30) 20-21,85.00,0.0,2022-09-30,20
718,(2022/09/30) 21-22,85.00,0.0,2022-09-30,21
719,(2022/09/30) 22-23,45.05,20.0,2022-09-30,22


In [90]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
5,39.06,9.4,2017-04-01,4
6,39.06,9.4,2017-04-01,5
7,39.06,9.4,2017-04-01,6
8,67.32,0.0,2017-04-01,7
9,67.32,0.0,2017-04-01,8
...,...,...,...,...
716,90.00,0.0,2022-09-30,19
717,85.00,0.0,2022-09-30,20
718,85.00,0.0,2022-09-30,21
719,45.05,20.0,2022-09-30,22


In [91]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2017-04-01,39.06,9.4,4,2017,4,1,5
2017-04-01,39.06,9.4,5,2017,4,1,5
2017-04-01,39.06,9.4,6,2017,4,1,5
2017-04-01,67.32,0.0,7,2017,4,1,5
2017-04-01,67.32,0.0,8,2017,4,1,5
...,...,...,...,...,...,...,...
2022-09-30,90.00,0.0,19,2022,9,30,4
2022-09-30,85.00,0.0,20,2022,9,30,4
2022-09-30,85.00,0.0,21,2022,9,30,4


In [92]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume     value  price_weight_av  price_max  price_min  \
year month day                                                            
2017 4     1     122.2  4773.132            39.06      67.32      39.06   
           2     225.6  8811.936            39.06      39.06      39.06   
           3      37.6  1468.656            39.06      67.32      39.06   
           4      37.6  1468.656            39.06      67.32      39.06   
           5      75.2  2937.312            39.06      67.32      39.06   
...                ...       ...              ...        ...        ...   
2022 9     26    160.0  7208.000            45.05      90.00      45.05   
           27    160.0  7208.000            45.05      90.00      45.05   
           28    160.0  7208.000            45.05      90.00      45.05   
           29     80.0  3604.000            45.05      90.00      45.05   
           30     80.0  3604.000            45.05      90.00      45.05   

                price_mean  power_max  power_min  power_mean       date  
year month day                                                           
2017 4     1        48.951        9.4        0.0    6.110000 2017-04-01  
           2        39.060        9.4        9.4    9.400000 2017-04-02  
           3        61.668        9.4        0.0    1.880000 2017-04-03  
           4        61.668        9.4        0.0    1.880000 2017-04-04  
           5        57.900        9.4        0.0    3.133333 2017-04-05  
...                    ...        ...        ...         ...        ...  
2022 9     26       72.725       20.0        0.0    6.666667 2022-09-26  
           27       72.725       20.0        0.0    6.666667 2022-09-27  
           28       72.725       20.0        0.0    6.666667 2022-09-28  
           29       78.260       20.0        0.0    4.000000 2022-09-29  
           30       78.260       20.0        0.0    4.000000 2022-09-30  

[1875 rows x 10 columns]

In [93]:
hourly['cat']='off'

if (pref=='dam' or pref=='idm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2017-04-01,39.06,9.4,4,2017,4,1,5,367.164,off
2017-04-01,39.06,9.4,5,2017,4,1,5,367.164,off
2017-04-01,39.06,9.4,6,2017,4,1,5,367.164,off
2017-04-01,67.32,0.0,7,2017,4,1,5,0.000,standard
2017-04-01,67.32,0.0,8,2017,4,1,5,0.000,standard
...,...,...,...,...,...,...,...,...,...
2022-09-30,90.00,0.0,19,2022,9,30,4,0.000,standard
2022-09-30,85.00,0.0,20,2022,9,30,4,0.000,standard
2022-09-30,85.00,0.0,21,2022,9,30,4,0.000,standard


In [94]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume     value  price_weight_av  price_max  price_min  \
year month day                                                            
2017 4     1     122.2  4773.132            39.06      67.32      39.06   
           2     225.6  8811.936            39.06      39.06      39.06   
           3      37.6  1468.656            39.06      67.32      39.06   
           4      37.6  1468.656            39.06      67.32      39.06   
           5      75.2  2937.312            39.06      67.32      39.06   
...                ...       ...              ...        ...        ...   
2022 9     26    160.0  7208.000            45.05      90.00      45.05   
           27    160.0  7208.000            45.05      90.00      45.05   
           28    160.0  7208.000            45.05      90.00      45.05   
           29     80.0  3604.000            45.05      90.00      45.05   
           30     80.0  3604.000            45.05      90.00      45.05   

                price_mean  power_max  power_min  power_mean       date  ...  \
year month day                                                           ...   
2017 4     1        48.951        9.4        0.0    6.110000 2017-04-01  ...   
           2        39.060        9.4        9.4    9.400000 2017-04-02  ...   
           3        61.668        9.4        0.0    1.880000 2017-04-03  ...   
           4        61.668        9.4        0.0    1.880000 2017-04-04  ...   
           5        57.900        9.4        0.0    3.133333 2017-04-05  ...   
...                    ...        ...        ...         ...        ...  ...   
2022 9     26       72.725       20.0        0.0    6.666667 2022-09-26  ...   
           27       72.725       20.0        0.0    6.666667 2022-09-27  ...   
           28       72.725       20.0        0.0    6.666667 2022-09-28  ...   
           29       78.260       20.0        0.0    4.000000 2022-09-29  ...   
           30       78.260       20.0        0.0    4.000000 2022-09-30  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2017 4     1                   NaN                 NaN       122.2   
           2                   NaN                 NaN       225.6   
           3                   NaN                 NaN        37.6   
           4                   NaN                 NaN        37.6   
           5                   NaN                 NaN        75.2   
...                            ...                 ...         ...   
2022 9     26                  NaN                 NaN       160.0   
           27                  NaN                 NaN       160.0   
           28                  NaN                 NaN       160.0   
           29                  NaN                 NaN        80.0   
           30                  NaN                 NaN        80.0   

                standard_volume  morning_volume  evening_volume  off_value  \
year month day                                                               
2017 4     1                0.0             NaN             NaN   4773.132   
           2                NaN             NaN             NaN   8811.936   
           3                0.0             NaN             NaN   1468.656   
           4                0.0             NaN             NaN   1468.656   
           5                0.0             NaN             NaN   2937.312   
...                         ...             ...             ...        ...   
2022 9     26               0.0             NaN             NaN   7208.000   
           27               0.0             NaN             NaN   7208.000   
           28               0.0             NaN             NaN   7208.000   
           29               0.0             NaN             NaN   3604.000   
           30               0.0             NaN             NaN   3604.000   

                standard_value  morning_value  evening_value  
year month

In [95]:
daily.to_csv(out + pref + '_daily.csv')

In [96]:
hourly.to_csv(out + pref + '_hourly.csv')

In [97]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2017-04-01,4,39.06,9.4,off
2017-04-01,5,39.06,9.4,off
2017-04-01,6,39.06,9.4,off
2017-04-01,7,67.32,0.0,standard
2017-04-01,8,67.32,0.0,standard
...,...,...,...,...
2022-09-30,19,90.00,0.0,standard
2022-09-30,20,85.00,0.0,standard
2022-09-30,21,85.00,0.0,standard


In [98]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [99]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,4,105.593333,4124.475600,61.6680,39.06,53.606050,9.400000,1.566667,4.561611,...,0.0,39.060000,39.06,NaN,NaN,NaN,9.4,NaN,0.000000,NaN
1,2017,5,107.096774,3970.077419,57.6780,37.07,50.891763,10.000000,1.612903,4.634409,...,0.0,37.070000,37.07,37.07,NaN,NaN,10.0,NaN,0.404040,NaN
2,2017,6,26.400000,1176.384000,68.3680,64.40,66.384000,4.400000,0.000000,1.173333,...,0.0,44.560000,44.56,NaN,NaN,NaN,4.4,NaN,0.000000,NaN
3,2017,7,273.333333,11540.133333,68.4040,42.22,54.766500,20.000000,6.666667,12.333333,...,0.0,42.220000,42.22,NaN,NaN,NaN,20.0,NaN,0.000000,NaN
4,2017,8,380.833333,22038.583333,75.4000,33.40,62.785417,20.000000,15.833333,17.201389,...,0.0,57.869365,33.40,85.90,NaN,NaN,20.0,NaN,15.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2022,5,0.000000,0.000000,113.7500,62.00,94.383065,0.000000,0.000000,0.000000,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,NaN
61,2022,6,318.000000,29418.480000,103.6120,36.66,81.780728,27.333333,2.666667,13.705556,...,0.0,92.510943,36.66,198.12,NaN,NaN,20.0,NaN,8.684211,NaN
62,2022,7,220.000000,9911.000000,107.3175,45.05,83.494637,20.000000,3.225806,9.645161,...,0.0,45.050000,45.05,NaN,NaN,NaN,20.0,NaN,0.000000,NaN
63,2022,8,203.870968,9184.387097,91.3000,45.05,76.889516,20.000000,2.580645,8.752688,...,0.0,45.050000,45.05,NaN,NaN,NaN,20.0,NaN,0.000000,NaN


In [100]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,4,105.593333,4124.475600,61.6680,39.06,53.606050,9.400000,1.566667,4.561611,...,0.0,39.060000,39.06,NaN,NaN,NaN,9.4,NaN,0.000000,NaN
1,2017,5,107.096774,3970.077419,57.6780,37.07,50.891763,10.000000,1.612903,4.634409,...,0.0,37.070000,37.07,37.07,NaN,NaN,10.0,NaN,0.404040,NaN
2,2017,6,26.400000,1176.384000,68.3680,64.40,66.384000,4.400000,0.000000,1.173333,...,0.0,44.560000,44.56,NaN,NaN,NaN,4.4,NaN,0.000000,NaN
3,2017,7,273.333333,11540.133333,68.4040,42.22,54.766500,20.000000,6.666667,12.333333,...,0.0,42.220000,42.22,NaN,NaN,NaN,20.0,NaN,0.000000,NaN
4,2017,8,380.833333,22038.583333,75.4000,33.40,62.785417,20.000000,15.833333,17.201389,...,0.0,57.869365,33.40,85.90,NaN,NaN,20.0,NaN,15.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2022,5,0.000000,0.000000,113.7500,62.00,94.383065,0.000000,0.000000,0.000000,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,NaN
61,2022,6,318.000000,29418.480000,103.6120,36.66,81.780728,27.333333,2.666667,13.705556,...,0.0,92.510943,36.66,198.12,NaN,NaN,20.0,NaN,8.684211,NaN
62,2022,7,220.000000,9911.000000,107.3175,45.05,83.494637,20.000000,3.225806,9.645161,...,0.0,45.050000,45.05,NaN,NaN,NaN,20.0,NaN,0.000000,NaN
63,2022,8,203.870968,9184.387097,91.3000,45.05,76.889516,20.000000,2.580645,8.752688,...,0.0,45.050000,45.05,NaN,NaN,NaN,20.0,NaN,0.000000,NaN


In [101]:
monthly=monthly.set_index(['year','month'])

In [102]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [103]:
monthly.to_csv(out + pref + '_monthly.csv')

In [104]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [105]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,off_price_mean,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,7684.890323,43.387500,95.136710,39.500000,71.125947,16.000000,2.838710,7.681720,39.500000,...,0.0,2.382316e+05,326.000000,386.000000,0.0,0.0,16.000000,0.571211,NaN,NaN
1,2,7449.539745,42.977699,86.248794,39.342128,68.048585,16.167376,2.289362,7.498180,39.342128,...,0.0,2.100770e+05,306.600000,349.400000,0.0,0.0,16.140000,0.000000,NaN,NaN
2,3,5414.446617,42.748169,88.964135,42.632481,67.065624,10.676692,2.105263,5.459900,42.632481,...,0.0,1.440243e+05,294.000000,315.600000,0.0,0.0,12.000000,0.165375,NaN,NaN
3,4,7611.368156,43.754000,76.921333,40.611667,60.609724,14.900000,2.927778,7.406565,40.611667,...,0.0,2.283410e+05,320.333333,371.666667,0.0,0.0,14.900000,0.995435,NaN,NaN
4,5,3876.361290,34.730000,81.900914,39.061667,63.180427,10.000000,1.827957,4.855735,39.061667,...,0.0,1.201672e+05,327.000000,385.000000,0.0,0.0,10.000000,0.341329,NaN,NaN
5,6,10044.841895,55.036227,110.299671,37.527763,76.990835,17.294737,2.105263,7.961711,37.527763,...,0.0,2.544693e+05,267.666667,316.333333,0.0,0.0,14.066667,1.447368,NaN,NaN
6,7,48466.706582,65.766733,129.198481,43.864177,91.495701,56.629114,9.367089,30.307954,43.864177,...,0.0,1.276290e+06,278.500000,325.500000,0.0,0.0,40.833333,15.400000,NaN,NaN
7,8,6330.061491,45.911542,110.970870,44.024720,83.186013,12.298137,2.329193,5.953675,44.024720,...,0.0,1.698566e+05,281.833333,333.500000,0.0,0.0,13.333333,2.500000,NaN,NaN
8,9,5956.745556,33.786000,76.404611,35.813333,61.296218,16.666667,2.333333,7.618519,35.813333,...,0.0,1.787024e+05,320.333333,371.666667,0.0,0.0,16.666667,0.000000,NaN,NaN
9,10,7327.478710,32.770000,78.739806,31.804000,61.550816,20.000000,2.967742,9.282796,31.804000,...,0.0,2.271518e+05,326.000000,386.000000,0.0,0.0,20.000000,0.000000,NaN,NaN


In [106]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [107]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price     volume        value
month hour                                   
1     0     39.500000  16.000000   694.200000
      1     39.500000  16.000000   694.200000
      2     39.500000  16.000000   694.200000
      3     39.500000  16.000000   694.200000
      4     39.500000  16.000000   694.200000
...               ...        ...          ...
12    19    96.102258   6.077419   313.427484
      20    89.343710   8.472581   419.588452
      21    89.343710   8.472581   419.588452
      22    45.815000  22.500000  1066.500000
      23    45.815000  22.500000  1066.500000

[288 rows x 3 columns]

In [108]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [109]:
yearly_calmonthly_hours=hourly.groupby(['year','month','hour']).mean()
yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)
yearly_calmonthly_hours

price     volume       value
year month hour                                  
2017 4     0     39.060000   9.400000  367.164000
           1     39.060000   9.400000  367.164000
           2     39.060000   9.400000  367.164000
           3     39.060000   9.400000  367.164000
           4     39.060000   9.400000  367.164000
...                    ...        ...         ...
2022 9     19    83.340000   2.666667  120.133333
           20    74.346667   5.333333  240.266667
           21    74.346667   5.333333  240.266667
           22    45.050000  20.000000  901.000000
           23    45.050000  20.000000  901.000000

[1560 rows x 3 columns]

In [110]:
yearly_calmonthly_hours.to_csv(out + pref + '_yearly_calmonthly_hours.csv')

In [111]:
from statsmodels.tsa.stattools import pacf

In [112]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.0,1.0,1.000000
1,0.577884,NaN,NaN,0.726903
2,-0.151192,NaN,NaN,0.002911
3,0.101344,NaN,NaN,0.061935
4,0.379980,NaN,NaN,0.262009
5,0.217199,NaN,NaN,-0.265546
6,0.065991,NaN,NaN,-0.112756
7,0.017970,NaN,NaN,0.135923
8,0.063891,NaN,NaN,-0.048078
9,-0.017047,NaN,NaN,-0.233700


In [113]:
pacfs.to_csv(out + pref + '_pacfs.csv')